# Introduction

This IPython notebook illustrates how to select the best learning based matcher. First, we need to import py_entitymatching package and other libraries as follows:

In [1]:
# Import py_entitymatching package
import py_entitymatching as em
import os
import pandas as pd

# Set the seed value 
seed = 0


# Preprocessing from RSupCon directory

In [2]:
import pandas as pd
import copy

path="~/contrastive-product-matching/data/raw/shs100k2_yt/"

def clean_string(text):
    import re
    pattern = re.compile('[^a-zA-Z0-9\s]')
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

A = pd.read_parquet(path + "table.parquet").rename({"title": "video_title"}, axis=1).drop_duplicates(
    subset=["yt_id"]
)
A.description = A.description.apply(clean_string)
B = copy.copy(A)

df_train = pd.read_csv(path + "train.csv")

def enrich(pairs):
    data = pd.merge(
        A.add_prefix("ltable_").rename({"ltable_yt_id": "ltable_id"}, axis=1),
        pd.merge(
            pairs[["ltable_id", "rtable_id", "label"]],
            B.add_prefix("rtable_").rename({"rtable_yt_id": "rtable_id"}, axis=1),
            on="rtable_id",
            how="left"
            ), 
        on="ltable_id",
        how="right"
    )
    # id formatting
    data["_id"] = range(0, len(data))
    
    # col reordering
    data = data[['_id', 'ltable_id', 'rtable_id', 'ltable_video_title', 
            'ltable_channel_name', 'ltable_description', 
            'rtable_video_title', 'rtable_channel_name', 
            'rtable_description', 'label']]

    return data
    
S = enrich(df_train)

datasets_dir = em.get_install_path() + os.sep + 'datasets'

path_A = datasets_dir + os.sep + 'shs100k2_yt_A.csv'
path_B = datasets_dir + os.sep + 'shs100k2_yt_B.csv'
path_labeled_data = datasets_dir + os.sep + 'shs100k2_yt_labeled.csv'

os.remove(path_A) if os.path.exists(path_A) else None
os.remove(path_B) if os.path.exists(path_B) else None
os.remove(path_labeled_data) if os.path.exists(path_labeled_data) else None

em.to_csv_metadata(A, path_A)
em.to_csv_metadata(B, path_B)
em.to_csv_metadata(S, path_labeled_data)


# reading in
A = em.read_csv_metadata(path_A, key='yt_id')
B = em.read_csv_metadata(path_B, key='yt_id')

S = em.read_csv_metadata(path_labeled_data, key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

# remove the files
os.remove(path_A) if os.path.exists(path_A) else None
os.remove(path_B) if os.path.exists(path_B) else None
os.remove(path_labeled_data) if os.path.exists(path_labeled_data) else None



In [ ]:

# Split S into I an J
IJ = em.split_train_test(S, train_proportion=0.5, random_state=0)
I = IJ['train']
J = IJ['test']


# Selecting the Best learning-based matcher 

This, typically involves the following steps:
1. Creating a set of learning-based matchers
2. Creating features
3. Extracting feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation
5. Debugging the matcher (and possibly repeat the above steps)

## Creating a set of learning-based matchers

First, we need to create a set of learning-based matchers. The following matchers are supported in Magellan: (1) decision tree, (2) random forest, (3) naive bayes, (4) svm, (5) logistic regression, and (6) linear regression.

In [ ]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')


## Creating features

Next, we need to create a set of features for the development set. Magellan provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [ ]:
# Generate a set of features
F = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)


We observe that there were 20 features generated. As a first step, lets say that we decide to use only 'year' related features.

In [ ]:
F.feature_name

0                       yt_id_yt_id_jac_qgm_3_qgm_3
1                   yt_id_yt_id_cos_dlm_dc0_dlm_dc0
2                   yt_id_yt_id_jac_dlm_dc0_dlm_dc0
3                                   yt_id_yt_id_mel
4                              yt_id_yt_id_lev_dist
5                               yt_id_yt_id_lev_sim
6                                   yt_id_yt_id_nmw
7                                    yt_id_yt_id_sw
8           video_title_video_title_jac_qgm_3_qgm_3
9       video_title_video_title_cos_dlm_dc0_dlm_dc0
10                      video_title_video_title_mel
11                 video_title_video_title_lev_dist
12                  video_title_video_title_lev_sim
13        channel_name_channel_name_jac_qgm_3_qgm_3
14    channel_name_channel_name_cos_dlm_dc0_dlm_dc0
15    channel_name_channel_name_jac_dlm_dc0_dlm_dc0
16                    channel_name_channel_name_mel
17               channel_name_channel_name_lev_dist
18                channel_name_channel_name_lev_sim
19          

## Extracting feature vectors

In this step, we extract feature vectors using the development set and the created features.

In [ ]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=F, 
                            attrs_after='label',
                            show_progress=False)


In [ ]:
# Display first few rows
H.head()

,_id,ltable_id,rtable_id,yt_id_yt_id_jac_qgm_3_qgm_3,yt_id_yt_id_cos_dlm_dc0_dlm_dc0,yt_id_yt_id_jac_dlm_dc0_dlm_dc0,yt_id_yt_id_mel,yt_id_yt_id_lev_dist,yt_id_yt_id_lev_sim,yt_id_yt_id_nmw,...,channel_name_channel_name_cos_dlm_dc0_dlm_dc0,channel_name_channel_name_jac_dlm_dc0_dlm_dc0,channel_name_channel_name_mel,channel_name_channel_name_lev_dist,channel_name_channel_name_lev_sim,channel_name_channel_name_nmw,channel_name_channel_name_sw,description_description_jac_qgm_3_qgm_3,description_description_cos_dlm_dc0_dlm_dc0,label
333,333,maLde3rhWnI,5OZfoFAs2u4,0.0,0.0,0.0,0.000000,11,0.000000,0.0,...,0.0,0.0,0.507937,14.0,0.000000,0.0,2.0,0.018072,0.000000,0
6392,6392,bXKwBvhd-ww,N2_k9Hctwm8,0.0,0.0,0.0,0.393939,11,0.000000,0.0,...,0.0,0.0,0.572249,16.0,0.157895,-5.0,1.0,0.006667,0.000000,0
4786,4786,3V9RLXOiDP8,7iUZNsV5M_I,0.0,0.0,0.0,0.000000,11,0.000000,0.0,...,0.0,0.0,0.536538,9.0,0.307692,-1.0,2.0,NaN,NaN,0
357,357,hxqKTWHXmWk,LsGygMw0-OQ,0.0,0.0,0.0,0.000000,11,0.000000,0.0,...,0.0,0.0,0.394444,12.0,0.000000,-2.0,1.0,0.035398,0.048113,0
9663,9663,lQJJzsMcUEc,Qkdj9VfUJxU,0.0,0.0,0.0,0.454545,10,0.090909,0.0,...,0.0,0.0,0.317460,10.0,0.166667,-3.0,2.0,NaN,NaN,0


In [ ]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.notnull(H))

True

We observe that the extracted feature vectors contain missing values. We have to impute the missing values for the learning-based matchers to fit the model correctly. For the purposes of this guide, we impute the missing value in a column with the mean of the values in that column. 

In [ ]:
# Impute feature vectors with the mean of the column values.
H = em.impute_table(H, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
                strategy='mean')


/home/hachmeier/miniconda3/envs/py_entitymatching/lib/python3.7/site-packages/py_entitymatching-0.4.1-py3.7-linux-x86_64.egg/py_entitymatching/matcher/matcherutils.py:224: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  imp.statistics_[pd.np.isnan(imp.statistics_)] = val_all_nans


## Selecting the best matcher using cross-validation

Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use 'precision' metric to select the best matcher.

In [ ]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
        k=5,
        target_attr='label', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

14086.05s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
14086.31s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
14086.50s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
14086.69s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
14086.88s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
14087.07s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
14087.25s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
14087.43s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
14087.63s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
/home/hachmeier/miniconda3/envs/py_entitymatching/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.738488,0.755438,0.746801
1,RF,0.928184,0.749415,0.829194
2,SVM,0.986667,0.093073,0.169871
3,LinReg,0.989327,0.580030,0.730647
4,LogReg,0.968274,0.566521,0.713736


In [ ]:
result['drill_down_cv_stats']['precision']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7efdbf4e8390>,5,0.714286,0.758170,0.742138,0.721088,0.756757,0.738488
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7efe5cf9ce10>,5,0.957265,0.957627,0.930233,0.869565,0.926230,0.928184
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7efe5cf9cdd0>,5,1.000000,0.933333,1.000000,1.000000,1.000000,0.986667
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7efe5ec52d10>,5,1.000000,0.978947,0.979592,1.000000,0.988095,0.989327
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7efe5ec52710>,5,0.975309,0.989362,0.950495,0.939024,0.987179,0.968274


In [ ]:
result['drill_down_cv_stats']['recall']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7efdbf4e8390>,5,0.738255,0.778523,0.751592,0.757143,0.751678,0.755438
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7efe5cf9ce10>,5,0.751678,0.758389,0.764331,0.714286,0.758389,0.749415
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7efe5cf9cdd0>,5,0.100671,0.093960,0.070064,0.100000,0.100671,0.093073
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7efe5ec52d10>,5,0.550336,0.624161,0.611465,0.557143,0.557047,0.580030
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7efe5ec52710>,5,0.530201,0.624161,0.611465,0.550000,0.516779,0.566521


In [ ]:
result['drill_down_cv_stats']['f1']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7efdbf4e8390>,5,0.726073,0.768212,0.746835,0.738676,0.754209,0.746801
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7efe5cf9ce10>,5,0.842105,0.846442,0.839161,0.784314,0.833948,0.829194
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7efe5cf9cdd0>,5,0.182927,0.170732,0.130952,0.181818,0.182927,0.169871
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7efe5ec52d10>,5,0.709957,0.762295,0.752941,0.715596,0.712446,0.730647
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7efe5ec52710>,5,0.686957,0.765432,0.744186,0.693694,0.678414,0.713736


### Debug X (Random Forest)

In [ ]:
# Split H into P and Q
PQ = em.split_train_test(H, train_proportion=0.5, random_state=0)
P = PQ['train']
Q = PQ['test']

In [ ]:
# Debug RF matcher using GUI
em.vis_debug_rf(rf, P, Q, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
        target_attr='label')


: 

In [ ]:
# Add a feature to do Jaccard on title + authors and add it to F

# Create a feature declaratively
sim = em.get_sim_funs_for_matching()
tok = em.get_tokenizers_for_matching()
feature_string = """jaccard(wspace((ltuple['title'] + ' ' + ltuple['authors']).lower()), 
                            wspace((rtuple['title'] + ' ' + rtuple['authors']).lower()))"""
feature = em.get_feature_fn(feature_string, sim, tok)

# Add feature to F
em.add_feature(F, 'jac_ws_title_authors', feature)

True

In [ ]:
# Convert I into feature vectors using updated F
H = em.extract_feature_vecs(I, 
                            feature_table=F, 
                            attrs_after='label',
                            show_progress=False)

In [ ]:
# Check whether the updated F improves X (Random Forest)
result = em.select_matcher([rf], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
        k=5,
        target_attr='label', metric_to_select_matcher='f1', random_state=0)
result['drill_down_cv_stats']['f1']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x10db02310>,5,0.974359,1.0,0.962963,0.933333,1.0,0.974131


In [ ]:
# Select the best matcher again using CV
result = em.select_matcher([dt, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
        k=5,
        target_attr='label', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,1.000000,1.000000,1.000000
1,RF,1.000000,0.950714,0.974131
2,SVM,1.000000,0.837418,0.907995
3,LinReg,1.000000,0.970330,0.984593
4,LogReg,0.985714,0.935330,0.958724


In [ ]:
result['drill_down_cv_stats']['f1']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x10db02990>,5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x10db02310>,5,0.974359,1.000000,0.962963,0.933333,1.000000,0.974131
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x10db02390>,5,0.947368,0.960000,0.782609,0.933333,0.916667,0.907995
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x10db020d0>,5,1.000000,1.000000,0.962963,1.000000,0.960000,0.984593
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x10db02210>,5,0.974359,0.962963,0.962963,0.933333,0.960000,0.958724
